In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
all = pd.read_csv("allcombR.csv")
all

,chrom,x,y,type
0,chr1,0065_CRUK_PC_0065//chromo//100136559,0065_CRUK_PC_0065//chromo//100136559,comb
1,chr1,0065_CRUK_PC_0065//chromo//100136559,0065_CRUK_PC_0065//chromo//101980197,comb
2,chr1,0065_CRUK_PC_0065//chromo//100136559,0065_CRUK_PC_0065//chromo//105306563,comb
3,chr1,0065_CRUK_PC_0065//chromo//100136559,0065_CRUK_PC_0065//chromo//105338362,comb
4,chr1,0065_CRUK_PC_0065//chromo//100136559,0065_CRUK_PC_0065//chromo//106066705,comb
...,...,...,...,...
32426520,chrY,CPCG0414//sv//22420135,CPCG0373//sv//22262633,comb
32426521,chrY,CPCG0414//sv//22420135,CPCG0373//sv//22273551,comb
32426522,chrY,CPCG0414//sv//22420135,CPCG0373//sv//22420151,comb
32426523,chrY,CPCG0414//sv//22420135,CPCG0414//sv//22234068,comb


Data preprocessing:

In [3]:
all[['donor1', 'label1', 'pos1']] = all.x.str.split("//",expand=True) 

In [4]:
all[['donor2', 'label2', 'pos2']] = all.y.str.split("//",expand=True) 

In [5]:
all.drop(["x", "y"], axis = 1, inplace = True)
all = all[['chrom', 'donor1', 'label1', 'pos1', 'donor2', 'label2', 'pos2', 'type']]

In [ ]:
all.to_csv("allcomb.csv", index = False)

In [ ]:
all = pd.read_csv("allcomb.csv")
all

In [6]:
all.groupby('type').count()

,chrom,donor1,label1,pos1,donor2,label2,pos2
type,,,,,,,
chromo,3516,3516,3516,3516,3516,3516,3516
comb,32415064,32415064,32415064,32415064,32415064,32415064,32415064
sv,7945,7945,7945,7945,7945,7945,7945


Filter combinations pairs where: pos2 > pos1

In [ ]:
df = all[all['pos2'] > all['pos1']]
df

In [ ]:
df.to_csv("allcomb_filt.csv", index = False)

# START

In [7]:
df = pd.read_csv("allcomb_filt.csv")

In [8]:
df.groupby('type').count()

,chrom,donor1,label1,pos1,donor2,label2,pos2
type,,,,,,,
chromo,3516,3516,3516,3516,3516,3516,3516
comb,16190310,16190310,16190310,16190310,16190310,16190310,16190310
sv,7945,7945,7945,7945,7945,7945,7945


Discard real rearrangements from combinations dataset (for easier work further):

In [9]:
dfc = df[(df['type'] == 'comb')]
dfc

,chrom,donor1,label1,pos1,donor2,label2,pos2,type
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb
...,...,...,...,...,...,...,...,...
16201765,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22262633,comb
16201766,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22273551,comb
16201767,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22420151,comb
16201769,chrY,CPCG0414,sv,22420135,0091_CRUK_PC_0091,sv,23481907,comb


### 1) The same donor or not?

In [10]:
def donor(df):
  return df.assign(donor = np.where(df['donor1'] == df['donor2'], 's', 'd'))

In [11]:
dfd = donor(dfc)
dfd

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s
...,...,...,...,...,...,...,...,...,...
16201765,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22262633,comb,d
16201766,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22273551,comb,d
16201767,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22420151,comb,d
16201769,chrY,CPCG0414,sv,22420135,0091_CRUK_PC_0091,sv,23481907,comb,d


### 2) What type of combination: label1 = label2 and what type of label?

In [12]:
def label(df):
  return df.assign(binbin = np.where((df.label1 == df.label2) & (df.label1 == "sv"), 'svsv',
                                     np.where((df.label1 == df.label2) & (df.label1 == "chromo"), 'chrchr',
                                              np.where((df.label1 != df.label2), "svchr", "other"))))

In [13]:
dfl = label(dfd)
dfl

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s,chrchr
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s,chrchr
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s,chrchr
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s,chrchr
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s,chrchr
...,...,...,...,...,...,...,...,...,...,...
16201765,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22262633,comb,d,svsv
16201766,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22273551,comb,d,svsv
16201767,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22420151,comb,d,svsv
16201769,chrY,CPCG0414,sv,22420135,0091_CRUK_PC_0091,sv,23481907,comb,d,svsv


In [14]:
dfl.groupby('binbin').count()

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor
binbin,,,,,,,,,
chrchr,1934302,1934302,1934302,1934302,1934302,1934302,1934302,1934302,1934302
svchr,6896400,6896400,6896400,6896400,6896400,6896400,6896400,6896400,6896400
svsv,7359608,7359608,7359608,7359608,7359608,7359608,7359608,7359608,7359608


Append clusters informations to chrchr combinations of different donors:

In [15]:
cluster2 = pd.read_csv("clusters.csv")

In [16]:
cluster2 = cluster2.rename(columns={'donor_id': 'donor2'})
cluster2 = cluster2.rename(columns={'Start_cl': 'start2', 'End_cl': 'end2'})
cluster1 = cluster2.rename(columns={'donor2': 'donor1'})
cluster1 = cluster1.rename(columns={'start2': 'start1', 'end2': 'end1'})

In [17]:
cluster1 = cluster1.drop_duplicates()
cluster2 = cluster2.drop_duplicates()
cluster2

,donor2,chrom,start2,end2
0,EOPC-01,chr12,112938845,115173998
1,EOPC-011,chr6,145595779,168829760
5,EOPC-018,chr2,193357450,199873274
9,EOPC-021,chr1,5717619,90237612
15,EOPC-021,chr2,55039051,149412025
...,...,...,...,...
3494,CPCG0189,chr6,8530760,12542096
3501,CPCG0249,chr6,96628890,124190550
3511,CPCG0251,chr16,73979029,80691272
3513,CPCG0251,chr21,42872694,46565039


Choose comb_d_chrchr:

In [18]:
dchr = dfl[(dfl['binbin'] == 'chrchr') & (dfl['donor'] == 'd')]
dchr

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin
33,chr1,0065_CRUK_PC_0065,chromo,100136559,0077_CRUK_PC_0077,chromo,148930966,comb,d,chrchr
34,chr1,0065_CRUK_PC_0065,chromo,100136559,0077_CRUK_PC_0077,chromo,154357201,comb,d,chrchr
35,chr1,0065_CRUK_PC_0065,chromo,100136559,0077_CRUK_PC_0077,chromo,158830075,comb,d,chrchr
36,chr1,0065_CRUK_PC_0065,chromo,100136559,0077_CRUK_PC_0077,chromo,158888945,comb,d,chrchr
37,chr1,0065_CRUK_PC_0065,chromo,100136559,0077_CRUK_PC_0077,chromo,163267817,comb,d,chrchr
...,...,...,...,...,...,...,...,...,...,...
16197908,chrX,EOPC-049,chromo,55385945,CPCG0098,chromo,76279999,comb,d,chrchr
16197909,chrX,EOPC-049,chromo,55385945,CPCG0098,chromo,81887734,comb,d,chrchr
16197910,chrX,EOPC-049,chromo,55385945,CPCG0098,chromo,81890767,comb,d,chrchr
16197911,chrX,EOPC-049,chromo,55385945,CPCG0098,chromo,87941674,comb,d,chrchr


In [19]:
dc1 = pd.merge(dchr, cluster1, on=['donor1', 'chrom'], how ='left')
dcl = pd.merge(dc1, cluster2, on=['donor2', 'chrom'], how ='left')

In [20]:
dcl = dcl[['chrom', 'donor1', 'label1', 'start1', 'end1', 'pos1', 'donor2', 'label2', 'start2', 'end2', 'pos2', 'type', 'donor', 'binbin']]
dcl

,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin
0,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,148930966,comb,d,chrchr
1,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,154357201,comb,d,chrchr
2,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158830075,comb,d,chrchr
3,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158888945,comb,d,chrchr
4,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,163267817,comb,d,chrchr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764607,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,76279999,comb,d,chrchr
1764608,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81887734,comb,d,chrchr
1764609,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81890767,comb,d,chrchr
1764610,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,87941674,comb,d,chrchr


### 3) Chromo-clusters are instersecting or not?
condition: ( Start_2 > End_1 or Start_1 > End_2 ) => non-intersecting

In [21]:
def intersect_clusters(df):
  return df.assign(intersect_cluster = np.where((df['start2'] > df['end1']) |
                                                (df['start1'] > df['end2']), 'noninter', 'inter'))

In [22]:
dcli = intersect_clusters(dcl)
dcli

,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin,intersect_cluster
0,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,148930966,comb,d,chrchr,noninter
1,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,154357201,comb,d,chrchr,noninter
2,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158830075,comb,d,chrchr,noninter
3,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158888945,comb,d,chrchr,noninter
4,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,163267817,comb,d,chrchr,noninter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764607,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,76279999,comb,d,chrchr,inter
1764608,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81887734,comb,d,chrchr,inter
1764609,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81890767,comb,d,chrchr,inter
1764610,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,87941674,comb,d,chrchr,inter


### 4) How many positions in combination are in clusters overlap region?

Finding clusters overlap region:

In [23]:
def overlap1(df):
  return df.assign(overlap_start = np.where((df.intersect_cluster == 'inter'), df[["start1", "start2"]].max(axis=1), ""))

In [24]:
def overlap2(df):
  return df.assign(overlap_end = np.where((df.intersect_cluster == 'inter'), df[["end1", "end2"]].min(axis=1), ""))

In [25]:
ov1 = overlap1(dcli)
over = overlap2(ov1)
over

,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin,intersect_cluster,overlap_start,overlap_end
0,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,148930966,comb,d,chrchr,noninter,,
1,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,154357201,comb,d,chrchr,noninter,,
2,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158830075,comb,d,chrchr,noninter,,
3,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,158888945,comb,d,chrchr,noninter,,
4,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,0077_CRUK_PC_0077,chromo,148930966,248810132,163267817,comb,d,chrchr,noninter,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764607,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,76279999,comb,d,chrchr,inter,52897784,55385946
1764608,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81887734,comb,d,chrchr,inter,52897784,55385946
1764609,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81890767,comb,d,chrchr,inter,52897784,55385946
1764610,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,87941674,comb,d,chrchr,inter,52897784,55385946


In [26]:
over['overlap_start'] = pd.to_numeric(over.overlap_start)
over['overlap_end'] = pd.to_numeric(over.overlap_end)
over['pos1'] = pd.to_numeric(over.pos1)
over['pos2'] = pd.to_numeric(over.pos2)
over['start1'] = pd.to_numeric(over.start1)
over['start2'] = pd.to_numeric(over.start2)
over['end1'] = pd.to_numeric(over.end1)
over['end2'] = pd.to_numeric(over.end2)
inter = over[(over['intersect_cluster'] == 'inter')]
inter

,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin,intersect_cluster,overlap_start,overlap_end
94,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,112183197,comb,d,chrchr,inter,75879779.0,120025710.0
95,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,112184970,comb,d,chrchr,inter,75879779.0,120025710.0
96,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,113934297,comb,d,chrchr,inter,75879779.0,120025710.0
97,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,113935981,comb,d,chrchr,inter,75879779.0,120025710.0
98,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,114239691,comb,d,chrchr,inter,75879779.0,120025710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764607,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,76279999,comb,d,chrchr,inter,52897784.0,55385946.0
1764608,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81887734,comb,d,chrchr,inter,52897784.0,55385946.0
1764609,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81890767,comb,d,chrchr,inter,52897784.0,55385946.0
1764610,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,87941674,comb,d,chrchr,inter,52897784.0,55385946.0


### Classify positions of combination according to the clusters overlap:
1) both pos are in overlap;
2) just one pos is in overlap;
3) zero positons are in overlap, but both position are located at the same cluster of one donor;
4) zero positions are in overlap and they are located at their own clusters.

In [27]:
c1 = ((inter.overlap_start <= inter.pos1) & (inter.overlap_start <= inter.pos2) &
       (inter.overlap_end >= inter.pos1) & (inter.overlap_end >= inter.pos2))

c2 = ((inter.overlap_start <= inter.pos1) & (inter.overlap_end >= inter.pos1) |
      (inter.overlap_start <= inter.pos2) & (inter.overlap_end >= inter.pos2))

inter.loc[:, 'over_count'] = np.select(condlist=[c1, c2], choicelist=['2', '1'], default='0')
inter

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin,intersect_cluster,overlap_start,overlap_end,over_count
94,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,112183197,comb,d,chrchr,inter,75879779.0,120025710.0,2
95,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,112184970,comb,d,chrchr,inter,75879779.0,120025710.0,2
96,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,113934297,comb,d,chrchr,inter,75879779.0,120025710.0,2
97,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,113935981,comb,d,chrchr,inter,75879779.0,120025710.0,2
98,chr1,0065_CRUK_PC_0065,chromo,75879779,120025710,100136559,CPCG0094,chromo,8589683,155821955,114239691,comb,d,chrchr,inter,75879779.0,120025710.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764607,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,76279999,comb,d,chrchr,inter,52897784.0,55385946.0,1
1764608,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81887734,comb,d,chrchr,inter,52897784.0,55385946.0,1
1764609,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,81890767,comb,d,chrchr,inter,52897784.0,55385946.0,1
1764610,chrX,EOPC-049,chromo,52897784,55385946,55385945,CPCG0098,chromo,26243883,91967097,87941674,comb,d,chrchr,inter,52897784.0,55385946.0,1


In [28]:
inter.groupby('over_count').count()

,chrom,donor1,label1,start1,end1,pos1,donor2,label2,start2,end2,pos2,type,donor,binbin,intersect_cluster,overlap_start,overlap_end
over_count,,,,,,,,,,,,,,,,,
0,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337
1,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112
2,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829


Combine overlap dataset (only comb_d_chrchr) with all other data: other types of combinations


In [29]:
over = pd.merge(over, inter, how = "left")
data = pd.merge(dfl, over, how = "left")
data

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s,chrchr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s,chrchr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s,chrchr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s,chrchr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s,chrchr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16190305,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22262633,comb,d,svsv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16190306,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22273551,comb,d,svsv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16190307,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22420151,comb,d,svsv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16190308,chrY,CPCG0414,sv,22420135,0091_CRUK_PC_0091,sv,23481907,comb,d,svsv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
data[['type','donor','binbin','intersect_cluster','over_count']] = data[['type','donor','binbin','intersect_cluster','over_count']].fillna('')
data["name"] = data[['type','donor','binbin','intersect_cluster','over_count']].apply(lambda x: '_'.join(x), axis=1)
data["name"] = data['name'].replace({"comb_d_svchr__": "comb_d_svchr", "comb_d_svsv__": "comb_d_svsv",
                            "comb_s_chrchr__": "comb_s_chrchr", "comb_s_svchr__": "comb_s_svchr", 
                            "comb_s_svsv__": "comb_s_svsv", "comb_d_chrchr_noninter_": "comb_d_chrchr_noninter"})
data

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count,name
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16190305,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22262633,comb,d,svsv,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_d_svsv
16190306,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22273551,comb,d,svsv,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_d_svsv
16190307,chrY,CPCG0414,sv,22234068,CPCG0373,sv,22420151,comb,d,svsv,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_d_svsv
16190308,chrY,CPCG0414,sv,22420135,0091_CRUK_PC_0091,sv,23481907,comb,d,svsv,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_d_svsv


Finally, total stats:

In [31]:
data.groupby(['name']).count()

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count
name,,,,,,,,,,,,,,,,,,
comb_d_chrchr_inter_0,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337
comb_d_chrchr_inter_1,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112
comb_d_chrchr_inter_2,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829
comb_d_chrchr_noninter,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,0,0,198334
comb_d_svchr,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,0,0,0,0,6866945,0,0,6866945
comb_d_svsv,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,0,0,0,0,7224840,0,0,7224840
comb_s_chrchr,169690,169690,169690,169690,169690,169690,169690,169690,169690,169690,0,0,0,0,169690,0,0,169690
comb_s_svchr,29455,29455,29455,29455,29455,29455,29455,29455,29455,29455,0,0,0,0,29455,0,0,29455
comb_s_svsv,134768,134768,134768,134768,134768,134768,134768,134768,134768,134768,0,0,0,0,134768,0,0,134768


In [32]:
sv = df[(df.type != 'comb')]
sv

,chrom,donor1,label1,pos1,donor2,label2,pos2,type
5,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,116599471,chromo
3184,chr1,0065_CRUK_PC_0065,chromo,105338362,0065_CRUK_PC_0065,chromo,119512733,chromo
4209,chr1,0065_CRUK_PC_0065,chromo,106066705,0065_CRUK_PC_0065,chromo,119512941,chromo
7217,chr1,0065_CRUK_PC_0065,chromo,119502745,0065_CRUK_PC_0065,chromo,120025709,chromo
11978,chr1,0065_CRUK_PC_0065,chromo,75879779,0065_CRUK_PC_0065,chromo,106709343,chromo
...,...,...,...,...,...,...,...,...
16201701,chrY,CPCG0334,sv,10054356,CPCG0334,sv,10056094,sv
16201718,chrY,CPCG0334,sv,10054869,CPCG0334,sv,10056684,sv
16201753,chrY,CPCG0373,sv,22234071,CPCG0373,sv,22420151,sv
16201756,chrY,CPCG0373,sv,22262633,CPCG0373,sv,22273551,sv


In [33]:
full = pd.merge(data, sv, how = 'outer')
full

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count,name
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16201766,chrY,CPCG0334,sv,10054356,CPCG0334,sv,10056094,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16201767,chrY,CPCG0334,sv,10054869,CPCG0334,sv,10056684,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16201768,chrY,CPCG0373,sv,22234071,CPCG0373,sv,22420151,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16201769,chrY,CPCG0373,sv,22262633,CPCG0373,sv,22273551,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
full[['donor','binbin','intersect_cluster','over_count']] = data[['donor','binbin','intersect_cluster','over_count']].fillna('')
full["name"] = full[['type','donor','binbin','intersect_cluster','over_count']].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
full["name"] = full['name'].replace({"comb_d_svchr__": "comb_d_svchr", "comb_d_svsv__": "comb_d_svsv",
                            "comb_s_chrchr__": "comb_s_chrchr", "comb_s_svchr__": "comb_s_svchr", 
                            "comb_s_svsv__": "comb_s_svsv", "comb_d_chrchr_noninter_": "comb_d_chrchr_noninter",
                                    "sv_nan_nan_nan_nan": "sv", "chromo_nan_nan_nan_nan": "chromo"})
full

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count,name
0,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,101980197,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
1,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105306563,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
2,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,105338362,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
3,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106066705,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
4,chr1,0065_CRUK_PC_0065,chromo,100136559,0065_CRUK_PC_0065,chromo,106709343,comb,s,chrchr,NaN,NaN,NaN,NaN,,NaN,NaN,,comb_s_chrchr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16201766,chrY,CPCG0334,sv,10054356,CPCG0334,sv,10056094,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sv
16201767,chrY,CPCG0334,sv,10054869,CPCG0334,sv,10056684,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sv
16201768,chrY,CPCG0373,sv,22234071,CPCG0373,sv,22420151,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sv
16201769,chrY,CPCG0373,sv,22262633,CPCG0373,sv,22273551,sv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sv


In [35]:
a = full.groupby(['name']).count()
a

,chrom,donor1,label1,pos1,donor2,label2,pos2,type,donor,binbin,start1,end1,start2,end2,intersect_cluster,overlap_start,overlap_end,over_count
name,,,,,,,,,,,,,,,,,,
chromo,3516,3516,3516,3516,3516,3516,3516,3516,0,0,0,0,0,0,0,0,0,0
comb_d_chrchr_inter_0,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337,110337
comb_d_chrchr_inter_1,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112,854112
comb_d_chrchr_inter_2,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829,601829
comb_d_chrchr_noninter,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,198334,0,0,198334
comb_d_svchr,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,6866945,0,0,0,0,6866945,0,0,6866945
comb_d_svsv,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,7224840,0,0,0,0,7224840,0,0,7224840
comb_s_chrchr,169690,169690,169690,169690,169690,169690,169690,169690,169690,169690,0,0,0,0,169690,0,0,169690
comb_s_svchr,29455,29455,29455,29455,29455,29455,29455,29455,29455,29455,0,0,0,0,29455,0,0,29455


In [ ]:
full.to_csv("full_combinations.csv", index = False)